In [34]:
%load_ext sql
%sql sqlite:///northwind.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [35]:
%%sql
SELECT CustomerID, COUNT(*) 
FROM Customers 
GROUP BY CustomerID 
HAVING COUNT(*) > 1;

 * sqlite:///northwind.db
Done.


CustomerID,COUNT(*)


In [36]:
%%sql
CREATE TABLE Customers_new (
    CustomerID TEXT PRIMARY KEY,
    CustomerName TEXT,
    ContactName TEXT,
    ContactTitle TEXT,
    Address TEXT,
    City TEXT,
    Region TEXT,
    PostalCode TEXT,
    Country TEXT,
    Phone TEXT
);


 * sqlite:///northwind.db
Done.


[]

In [37]:
%%sql
INSERT INTO Customers_new (
    CustomerID,
    CustomerName,
    ContactName,
    ContactTitle,
    Address,
    City,
    Region,
    PostalCode,
    Country,
    Phone
)
SELECT
    CustomerID,
    CustomerName,
    ContactName,
    ContactTitle,
    Address,
    City,
    Region,
    PostalCode,
    Country,
    Phone
FROM Customers;


 * sqlite:///northwind.db
93 rows affected.


[]

In [38]:
%%sql
DROP TABLE Customers;

 * sqlite:///northwind.db
Done.


[]

In [39]:
%%sql
ALTER TABLE Customers_new RENAME TO Customers;

 * sqlite:///northwind.db
(sqlite3.OperationalError) error in view Customer and Suppliers by City: no such table: main.Customers
[SQL: ALTER TABLE Customers_new RENAME TO Customers;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [40]:
%%sql
SELECT name
FROM sqlite_master
WHERE type = 'view' AND sql LIKE '%Customers%';

 * sqlite:///northwind.db
Done.


name
Customer and Suppliers by City
Invoices
Orders Qry
Quarterly Orders
Sales Totals by Amount


In [41]:
%%sql
DROP VIEW "Customer and Suppliers by City";
DROP VIEW "Invoices";
DROP VIEW "Orders Qry";
DROP VIEW "Quarterly Orders";
DROP VIEW "Sales Totals by Amount";

 * sqlite:///northwind.db
Done.
Done.
Done.
Done.
Done.


[]

In [42]:
%%sql
ALTER TABLE Customers_new RENAME TO Customers;

 * sqlite:///northwind.db
Done.


[]

In [43]:
%%sql
CREATE VIEW "Customer and Suppliers by City" AS
SELECT City, CompanyName, ContactName, 'Customers' AS Relationship
FROM Customers
UNION
SELECT City, CompanyName, ContactName, 'Suppliers'
FROM Suppliers;

 * sqlite:///northwind.db
Done.


[]

In [44]:
%%sql
CREATE VIEW "Invoices" AS
SELECT Orders.OrderID, Customers.CustomerName, Customers.ContactName, 
       Orders.OrderDate, Orders.ShipName, Orders.ShipAddress, 
       Orders.ShipCity, Orders.ShipRegion, Orders.ShipPostalCode, 
       Orders.ShipCountry, OrderDetails.ProductID, Products.ProductName, 
       OrderDetails.UnitPrice, OrderDetails.Quantity, 
       OrderDetails.Discount, 
       (OrderDetails.UnitPrice * Quantity * (1 - Discount)) AS ExtendedPrice
FROM Customers
JOIN Orders ON Customers.CustomerID = Orders.CustomerID
JOIN OrderDetails ON Orders.OrderID = OrderDetails.OrderID
JOIN Products ON OrderDetails.ProductID = Products.ProductID;

 * sqlite:///northwind.db
Done.


[]

In [45]:
%%sql
CREATE VIEW "Orders Qry" AS
SELECT Orders.OrderID, Orders.CustomerID, Customers.CustomerName, 
       Orders.EmployeeID, Orders.OrderDate, Orders.RequiredDate, 
       Orders.ShippedDate, Orders.ShipVia, Orders.Freight, 
       Orders.ShipName, Orders.ShipAddress, Orders.ShipCity, 
       Orders.ShipRegion, Orders.ShipPostalCode, Orders.ShipCountry
FROM Orders
JOIN Customers ON Orders.CustomerID = Customers.CustomerID;


 * sqlite:///northwind.db
Done.


[]

In [46]:
%%sql
CREATE VIEW "Quarterly Orders" AS
SELECT Customers.CustomerName, Orders.OrderID, 
       Orders.OrderDate, Orders.ShippedDate
FROM Customers
JOIN Orders ON Customers.CustomerID = Orders.CustomerID
WHERE Orders.OrderDate BETWEEN '1997-01-01' AND '1997-03-31';


 * sqlite:///northwind.db
Done.


[]

In [47]:
%%sql
CREATE VIEW "Sales Totals by Amount" AS
SELECT Customers.CustomerName, Orders.OrderID, 
       SUM((UnitPrice * Quantity) * (1 - Discount)) AS SaleAmount, 
       Orders.OrderDate
FROM Orders
JOIN Customers ON Orders.CustomerID = Customers.CustomerID
JOIN OrderDetails ON Orders.OrderID = OrderDetails.OrderID
GROUP BY Customers.CustomerName, Orders.OrderID
HAVING SaleAmount > 2500;

 * sqlite:///northwind.db
Done.


[]

In [48]:
%%sql
DROP VIEW "Sales Totals by Amount";

 * sqlite:///northwind.db
Done.


[]

In [49]:
%%sql
CREATE VIEW "Sales Totals by Amount" AS
SELECT 
    Customers.CompanyName AS CustomerName,
    Orders.OrderID, 
    SUM((UnitPrice * Quantity) * (1 - Discount)) AS SaleAmount, 
    Orders.OrderDate
FROM Orders
JOIN Customers ON Orders.CustomerID = Customers.CustomerID
JOIN OrderDetails ON Orders.OrderID = OrderDetails.OrderID
GROUP BY Customers.CompanyName, Orders.OrderID
HAVING SaleAmount > 2500;


 * sqlite:///northwind.db
Done.


[]

In [50]:
%%sql
SELECT * FROM "Sales Totals by Amount" LIMIT 10;


 * sqlite:///northwind.db
(sqlite3.OperationalError) no such table: main.OrderDetails
[SQL: SELECT * FROM "Sales Totals by Amount" LIMIT 10;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
